<a href="https://colab.research.google.com/github/NayanaMarques/ANN_2021/blob/master/Projeto_Nayana_RN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [475]:
#Projeto 1: Redes Neurais - Professor Ubirajara Filho
#Nayana G. Marques Silva
#Artigo: Comparison of neural network application for fuzzy and ANFIS approaches for multi-criteria decision making problems (2014)
#
#Definição de bibliotecas
#
import numpy as np #biblioteca numérica Phyton
import pandas as pd #biblioteca para manipulação de arquivos
import matplotlib.pyplot as plt #biblioteca para gráficos no Phyton
from sklearn.neural_network import MLPRegressor #biblioteca para classificação e regressão (regressão - MLPRegressor)
from sklearn.metrics import mean_squared_error #biblioteca para soma dos quadrados dos desvios
from sklearn.metrics import r2_score #biblioteca para avaliação do ajuste a partir do coeficiente de determinação
from google.colab import files #biblioteca para carregar arquivos no Phyton

In [476]:
!git clone https://github.com/NayanaMarques/ANN_2021
%cd ANN_2021

Cloning into 'ANN_2021'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 49 (delta 23), reused 9 (delta 3), pack-reused 0
Unpacking objects: 100% (49/49), done.
/content/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021


In [477]:
# leitura dos dados
df=pd.read_csv("Dados de entrada.csv",delimiter=';')
print(df.head(34))

    Qual  Deliv  Tech  Price  Loc  Supplier Score Train or Test
0    3.0    6.0   3.5    4.0  1.5            0.05      Training
1    3.5    3.5   6.0    6.0  1.5            0.05          Test
2    3.0    4.0   5.0    4.0  7.0            0.07      Training
3    7.0    6.0   4.5    6.0  3.0            0.10          Test
4    6.5    6.5   6.5    6.5  5.5            0.11      Training
5    5.0    4.0   5.5    4.0  5.0            0.11          Test
6    5.0    6.0   6.0    5.5  2.5            0.12      Training
7    3.5    5.0   6.0    6.5  3.5            0.13          Test
8    5.5    4.5   6.0    4.5  7.0            0.15      Training
9    5.0    6.5   3.5    3.5  8.0            0.16          Test
10   5.5    6.5   5.5    7.0  5.5            0.19      Training
11   5.0    7.0   5.5    7.0  5.5            0.19          Test
12   4.0    6.0   7.0    5.0  9.0            0.19      Training
13   2.0    6.5   2.0    8.0  4.0            0.22          Test
14   5.5    7.5   5.5    3.5  7.5       

In [478]:
# Separa as colunas de interesse, imprimindo todas as linhas e as colunas indicadas no Index (lista)
Index=[0,1,2,3,4,5]
DS=[]
DS=df.iloc[:,Index].copy()
DS.head(3)

,Qual,Deliv,Tech,Price,Loc,Supplier Score
0,3.0,6.0,3.5,4.0,1.5,0.05
1,3.5,3.5,6.0,6.0,1.5,0.05
2,3.0,4.0,5.0,4.0,7.0,0.07


In [479]:
# funcao_normatiza_dados - normalizar dados (mesma ordem de grandeza)
def funcao_normatiza_dados(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [480]:
# funcao_dados_originais - retorna os dados a forma original
def funcao_dados_originais(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [481]:
# separa os dados em normatizados e originais
DAT_2=[]
DAT_2=DS.copy()
DAT=funcao_normatiza_dados (DS)

In [482]:
DATA.head(10) # conferência dos dados (entre -1 e 1)

,Qual,Deliv,Tech,Price,Loc,Supplier Score
0,-0.692308,-0.090909,-0.571429,-0.454545,-1.000000,-1.000000
1,-0.538462,-1.000000,0.142857,0.272727,-1.000000,-1.000000
2,-0.692308,-0.818182,-0.142857,-0.454545,0.466667,-0.932203
3,0.538462,-0.090909,-0.285714,0.272727,-0.600000,-0.830508
4,0.384615,0.090909,0.285714,0.454545,0.066667,-0.796610
5,-0.076923,-0.818182,0.000000,-0.454545,-0.066667,-0.796610
6,-0.076923,-0.090909,0.142857,0.090909,-0.733333,-0.762712
7,-0.538462,-0.454545,0.142857,0.454545,-0.466667,-0.728814
8,0.076923,-0.636364,0.142857,-0.272727,0.466667,-0.661017
9,-0.076923,0.090909,-0.571429,-0.636364,0.733333,-0.627119


In [483]:
# Separação dos dados: treino e teste (75% para treino e 25% para teste) - sample (randômico) - drop (não usar dados de treino para teste)
train=DAT.sample(frac=0.8,random_state=None) 
test=DAT.drop(train.index)
train.head(4) #print de checagem 

,Qual,Deliv,Tech,Price,Loc,Supplier Score
7,-0.538462,-0.454545,0.142857,0.454545,-0.466667,-0.728814
14,0.076923,0.454545,0.000000,-0.636364,0.600000,-0.389831
16,-0.384615,-0.818182,0.428571,0.454545,0.200000,-0.322034
3,0.538462,-0.090909,-0.285714,0.272727,-0.600000,-0.830508


In [484]:
test.tail(4) #print de checagem 

,Qual,Deliv,Tech,Price,Loc,Supplier Score
25,0.538462,0.090909,0.428571,-0.090909,0.733333,0.423729
26,0.230769,0.272727,0.142857,-0.090909,0.200000,0.457627
29,0.846154,1.000000,0.714286,-0.454545,-0.066667,0.830508
30,1.000000,0.272727,0.285714,-0.454545,0.466667,0.898305


In [485]:
# Definição das variáveis independentes (treino e teste) - Colunas 0 a 4
x_train=train.iloc[:,[0,1,2,3,4]]
x_test=test.iloc[:,[0,1,2,3,4]]
X_2=DAT_2.iloc[:,[0,1,2,3,4]]

In [486]:
# Definição da variável dependente - Coluna 5
y_train=train.iloc[:,5]
y_test=test.iloc[:,5]
Y_2=DAT_2.iloc[:,5]

In [487]:
# Definindo a rede neural - não foi utilizada a do artigo e sim a rede dada em aula (MLPRegressor do sklearn)
# solver: lbfgs (poucos dados) 
# função de ativação: tangente hiperbólica 
# alpha: pequeno (erro)
# Hidden_layer_size: 2 camadas ocultas com 5 neurônios cada
# random_state: dados randômicos
supplierscore = MLPRegressor(solver='lbfgs',activation='tanh',alpha=1e-5,max_iter=500, hidden_layer_sizes=(5, 5),random_state=1)

In [488]:
# Treinamento da rede neural construída
supplierscore.fit(x_train, y_train)

MLPRegressor(activation='tanh', alpha=1e-05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(5, 5), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=500,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [489]:
# Validação treino e teste
y_calc_train=supplierscore.predict(x_train)
y_calc_test=supplierscore.predict(x_test)

In [ ]:
# Conversão dados normatizados para a forma original (não normalizado)
y_calc_train=funcao_dados_originais(y_calc_train,Y_2)
y_calc_test=funcao_dados_originais(y_calc_test,Y_2)

In [490]:
# recuperando dados originais 
test=[]
train=[]
train=funcao_dados_originais(y_train,Y_2)
test=funcao_dados_originais(y_test,Y_2)

KeyError: ignored

In [ ]:
# Plotar gráfico
supplierscore_obs_train=[]
supplierscore_calc_train=[]
supplierscore_calc_train=y_calc_train['Supplie Score'].copy()
supplierscore_obs_train=train['Supplie Score'].copy()
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.scatter(supplierscore_obs_test,supplierscore_calc_test, s=20, c='b', marker="o", label='teste')
ax1.scatter(supplierscore_obs_train,supplierscore_calc_train, s=20, c='r', marker="o", label='treino')
plt.legend(loc='upper left')
plt.ylabel('Supplie Score Calculado')
plt.xlabel('Supplie Observado')
plt.show()

In [ ]:
mse=mean_squared_error(supplierscore_obs_test,supplierscore_calc_test)
print("MSE teste=",mse)
R2=r2_score(supplierscore_obs_test,supplierscore_calc_test)
print("R^2 teste=",R2)

In [ ]:
mse=mean_squared_error(supplierscore_obs_train,supplierscore_calc_train)
print("MSE treino=",mse)
R2=r2_score(supplierscore_obs_train,supplierscore_calc_train)
print("R^2 treino=",R2)